In [3]:
import tensorflow as tf
import datetime
import os
import numpy as np
import ctypes
from PIL import Image

In [4]:
#### ------------------------------------------- ####
#### ------------- LOAD THE DATA --------------- ####
#### ------------------------------------------- ####

In [5]:
MAIN_DATASET_FOLDER = "..\\dataset\\"

TEST_SUBFOLDER = os.path.join(MAIN_DATASET_FOLDER, "test")
TRAIN_SUBFOLDER = os.path.join(MAIN_DATASET_FOLDER, "train")

TEST_ESPAGNE_FOLDER = os.path.join(TEST_SUBFOLDER, "espagne")
TEST_FRANCE_FOLDER = os.path.join(TEST_SUBFOLDER, "france")
TEST_JAPON_FOLDER = os.path.join(TEST_SUBFOLDER, "japon")

TRAIN_ESPAGNE_FOLDER = os.path.join(TRAIN_SUBFOLDER, "espagne")
TRAIN_FRANCE_FOLDER = os.path.join(TRAIN_SUBFOLDER, "france")
TRAIN_JAPON_FOLDER = os.path.join(TRAIN_SUBFOLDER, "japon")


In [6]:
def fill_x_and_y_with_images_and_labels(folder, classe, dataset):

    for file in os.listdir(folder):

        file_path = os.path.join(folder, file)
        image = Image.open(file_path)
        image = image.resize((64, 64))
        if image.mode in ("RGB", "P"): image = image.convert("RGBA")
        im_arr = np.array(image).flatten()
        im_arr = im_arr / 255.0

        completed_dataset = {}
        completed_dataset["value"] = im_arr
        completed_dataset["classe"] = classe

        dataset.append(completed_dataset.copy())

In [7]:
def import_dataset(folder, classe):

    dataset = []

    fill_x_and_y_with_images_and_labels(folder, classe, dataset)

    return dataset

In [8]:
x_test = []
y_test = []

x_train = []
y_train = []

class_espagne_expected_outputs = [1, 0, 0]
dataset_test_for_espagne = import_dataset(TEST_ESPAGNE_FOLDER, class_espagne_expected_outputs)
dataset_train_for_espagne = import_dataset(TRAIN_ESPAGNE_FOLDER, class_espagne_expected_outputs)

class_france_expected_outputs = [0, 1, 0]
dataset_test_for_france = import_dataset(TEST_FRANCE_FOLDER, class_france_expected_outputs)
dataset_train_for_france = import_dataset(TRAIN_FRANCE_FOLDER, class_france_expected_outputs)

class_japon_expected_outputs = [0, 0, 1]
dataset_test_for_japon = import_dataset(TEST_JAPON_FOLDER, class_japon_expected_outputs)
dataset_train_for_japon = import_dataset(TRAIN_JAPON_FOLDER, class_japon_expected_outputs)

final_dataset_test = dataset_test_for_espagne + dataset_test_for_france + dataset_test_for_japon
final_dataset_train = dataset_train_for_espagne + dataset_train_for_france + dataset_train_for_japon

In [9]:
for value in final_dataset_test:

    the_image = value["value"].tolist()
    the_classe = value["classe"]

    x_test.append(the_image)
    y_test.append(the_classe)

for value in final_dataset_train:

    the_image = value["value"].tolist()
    the_classe = value["classe"]

    x_train.append(the_image)
    y_train.append(the_classe)

dataset_inputs = x_train
dataset_outputs = y_train

initiate_dataset_inputs_size = len(dataset_inputs[0])
initiate_dataset_inputs_type = ctypes.c_float * initiate_dataset_inputs_size

classes_initiate_dataset_inputs_size = initiate_dataset_inputs_size * 2

In [10]:
inputs_result = []

for sublist in dataset_inputs:
    for item in sublist:
        inputs_result.append(item)

dataset_inputs = inputs_result

outputs_result = []

for sublist in dataset_outputs:
    for item in sublist:
        outputs_result.append(item)

dataset_outputs = outputs_result

arr = [initiate_dataset_inputs_size, classes_initiate_dataset_inputs_size, 3]
arr_size = len(arr)
arr_type = ctypes.c_int * arr_size

In [11]:
# Convert train/test to array
x_train = np.array(x_train)
y_train = np.array(y_train)
x_test = np.array(x_test)
y_test = np.array(y_test)

In [12]:
#### ------------------------------------------- ####
#### ------------LOAD MODELS---------------------#### 
#### ------------------------------------------- ####

In [13]:
model_pmc_a = tf.keras.models.load_model("../models/keras_models/model_a.h5")

AttributeError: module 'tensorflow' has no attribute 'keras'

In [ ]:
model_pmc_b = tf.keras.models.load_model("../models/keras_models/model_b.h5")

In [13]:
model_pmc_c = tf.keras.models.load_model("../models/keras_models/model_c.h5")

In [14]:
model_pmc_d = tf.keras.models.load_model("../models/keras_models/model_d.h5")

In [15]:
#### ------------------------------------------- ####
#### ------ DEFINE THE TRAINING ALGORITHM ------ #### 
#### ------------------------------------------- ####

In [64]:
OPTIMIZER = tf.keras.optimizers.SGD(learning_rate=1e-3)
LOSS = tf.keras.losses.CategoricalCrossentropy(from_logits=False)
METRICS = [
    tf.keras.metrics.CategoricalAccuracy(name='acc'),
]

In [65]:
model_pmc_a.compile(
    optimizer=OPTIMIZER,
    loss=LOSS,
    metrics=METRICS
)

model_pmc_b.compile(
    optimizer=OPTIMIZER,
    loss=LOSS,
    metrics=METRICS
)

model_pmc_c.compile(
    optimizer=OPTIMIZER,
    loss=LOSS,
    metrics=METRICS
)

model_pmc_d.compile(
    optimizer=OPTIMIZER,
    loss=LOSS,
    metrics=METRICS
)

In [66]:
#### ------------------------------------------- ####
#### ------------- TRAIN MODEL (A) ------------- #### 
#### ------------------------------------------- ####

In [67]:
tf.keras.backend.clear_session()

In [68]:
# Callbacks for saving the checkpoints at every epoch.
checkpoint_cb = tf.keras.callbacks.ModelCheckpoint('../models/keras_models/model_a.h5', save_best_only=True)

In [69]:
tensorboard_cb = tf.keras.callbacks.TensorBoard(
    log_dir="../models/keras_models/keras_logs/" + datetime.datetime.now().strftime("model_a_%Y%m%d-%H%M%S"), 
    histogram_freq=1
)

history_a = model_pmc_a.fit(
    x=x_train, 
    y=y_train, 
    epochs=100, 
    validation_data=(x_test, y_test), 
    callbacks=[
        tensorboard_cb,
        checkpoint_cb,
        
        
    ]

)

Epoch 1/100
19/19 [==============================] - 1s 59ms/step - loss: 0.0964 - acc: 0.9933 - val_loss: 0.6524 - val_acc: 0.7667
Epoch 2/100
19/19 [==============================] - 0s 4ms/step - loss: 0.0984 - acc: 0.9950 - val_loss: 0.6507 - val_acc: 0.7417
Epoch 3/100
19/19 [==============================] - 0s 4ms/step - loss: 0.0951 - acc: 0.9950 - val_loss: 0.6360 - val_acc: 0.7167
Epoch 4/100
19/19 [==============================] - 0s 4ms/step - loss: 0.0961 - acc: 0.9967 - val_loss: 0.6241 - val_acc: 0.7417
Epoch 5/100
19/19 [==============================] - 0s 4ms/step - loss: 0.0986 - acc: 0.9950 - val_loss: 0.6370 - val_acc: 0.7250
Epoch 6/100
19/19 [==============================] - 0s 4ms/step - loss: 0.0968 - acc: 0.9900 - val_loss: 0.6297 - val_acc: 0.7583
Epoch 7/100
19/19 [==============================] - 0s 4ms/step - loss: 0.0954 - acc: 0.9917 - val_loss: 0.6475 - val_acc: 0.7667
Epoch 8/100
19/19 [==============================] - 0s 4ms/step - loss: 0.0953 - 

In [70]:
history_a.history.keys()

dict_keys(['loss', 'acc', 'val_loss', 'val_acc'])

In [71]:
#### ------------------------------------------- ####
#### ------------- TRAIN MODEL (B) ------------- #### 
#### ------------------------------------------- ####

In [72]:
tf.keras.backend.clear_session()

In [73]:
# Callbacks for saving the checkpoints at every epoch.
checkpoint_cb = tf.keras.callbacks.ModelCheckpoint('../models/keras_models/model_b.h5', save_best_only=True)

In [74]:
tensorboard_cb = tf.keras.callbacks.TensorBoard(
    log_dir="../models/keras_models/keras_logs/" + datetime.datetime.now().strftime("model_b_%Y%m%d-%H%M%S"), 
    histogram_freq=1
)

history_b = model_pmc_b.fit(
    x=x_train, 
    y=y_train, 
    epochs=100, 
    validation_data=(x_test, y_test), 
    callbacks=[
        tensorboard_cb,
        checkpoint_cb,
    ]
)

Epoch 1/100
19/19 [==============================] - 1s 59ms/step - loss: 0.0768 - acc: 0.9542 - val_loss: 0.6807 - val_acc: 0.7333
Epoch 2/100
19/19 [==============================] - 0s 4ms/step - loss: 0.0727 - acc: 0.9933 - val_loss: 0.6976 - val_acc: 0.7417
Epoch 3/100
19/19 [==============================] - 0s 4ms/step - loss: 0.0780 - acc: 0.9950 - val_loss: 0.7346 - val_acc: 0.7417
Epoch 4/100
19/19 [==============================] - 0s 4ms/step - loss: 0.0769 - acc: 0.9933 - val_loss: 0.6897 - val_acc: 0.7250
Epoch 5/100
19/19 [==============================] - 0s 4ms/step - loss: 0.0740 - acc: 0.9967 - val_loss: 0.6677 - val_acc: 0.7417
Epoch 6/100
19/19 [==============================] - 0s 4ms/step - loss: 0.0775 - acc: 0.9900 - val_loss: 0.7385 - val_acc: 0.7417
Epoch 7/100
19/19 [==============================] - 0s 4ms/step - loss: 0.0814 - acc: 0.9917 - val_loss: 0.7102 - val_acc: 0.7417
Epoch 8/100
19/19 [==============================] - 0s 4ms/step - loss: 0.0775 - 

In [75]:
history_b.history.keys()

dict_keys(['loss', 'acc', 'val_loss', 'val_acc'])

In [76]:
#### ------------------------------------------- ####
#### ------------- TRAIN MODEL (C) ------------- #### 
#### ------------------------------------------- ####

In [77]:
tf.keras.backend.clear_session()

In [78]:
# Callbacks for saving the checkpoints at every epoch.
checkpoint_cb = tf.keras.callbacks.ModelCheckpoint('../models/keras_models/model_c.h5', save_best_only=True)

In [79]:
tensorboard_cb = tf.keras.callbacks.TensorBoard(
    log_dir="../models/keras_models/keras_logs/" + datetime.datetime.now().strftime("model_c_%Y%m%d-%H%M%S"), 
    histogram_freq=1
)

history_c = model_pmc_c.fit(
    x=x_train, 
    y=y_train, 
    epochs=100, 
    validation_data=(x_test, y_test), 
    callbacks=[
        tensorboard_cb,
        checkpoint_cb,
    ]
)

Epoch 1/100
19/19 [==============================] - 1s 60ms/step - loss: 0.0913 - acc: 0.9542 - val_loss: 0.5653 - val_acc: 0.7833
Epoch 2/100
19/19 [==============================] - 0s 5ms/step - loss: 0.0958 - acc: 0.9867 - val_loss: 0.5463 - val_acc: 0.7750
Epoch 3/100
19/19 [==============================] - 0s 5ms/step - loss: 0.0975 - acc: 0.9883 - val_loss: 0.5647 - val_acc: 0.7833
Epoch 4/100
19/19 [==============================] - 0s 5ms/step - loss: 0.0920 - acc: 0.9900 - val_loss: 0.5804 - val_acc: 0.7917
Epoch 5/100
19/19 [==============================] - 0s 5ms/step - loss: 0.0872 - acc: 0.9933 - val_loss: 0.6010 - val_acc: 0.7917
Epoch 6/100
19/19 [==============================] - 0s 14ms/step - loss: 0.0878 - acc: 0.9917 - val_loss: 0.5798 - val_acc: 0.8000
Epoch 7/100
19/19 [==============================] - 0s 5ms/step - loss: 0.0871 - acc: 0.9933 - val_loss: 0.5655 - val_acc: 0.7750
Epoch 8/100
19/19 [==============================] - 0s 4ms/step - loss: 0.0859 -

In [80]:
history_c.history.keys()

dict_keys(['loss', 'acc', 'val_loss', 'val_acc'])

In [81]:
#### ------------------------------------------- ####
#### ------------- TRAIN MODEL (D) ------------- #### 
#### ------------------------------------------- ####

In [82]:
tf.keras.backend.clear_session()

In [83]:
# Callbacks for saving the checkpoints at every epoch.
checkpoint_cb = tf.keras.callbacks.ModelCheckpoint('../models/keras_models/model_d.h5', save_best_only=True)

In [84]:
tensorboard_cb = tf.keras.callbacks.TensorBoard(
    log_dir="../models/keras_models/keras_logs/" + datetime.datetime.now().strftime("model_d_%Y%m%d-%H%M%S"), 
    histogram_freq=1
)

history_d = model_pmc_d.fit(
    x=x_train, 
    y=y_train, 
    epochs=100, 
    validation_data=(x_test, y_test), 
    callbacks=[
        tensorboard_cb,
        checkpoint_cb,
    ]
)

Epoch 1/100
19/19 [==============================] - 1s 60ms/step - loss: 0.0809 - acc: 0.9528 - val_loss: 0.6666 - val_acc: 0.7333
Epoch 2/100
19/19 [==============================] - 0s 5ms/step - loss: 0.0832 - acc: 0.9850 - val_loss: 0.8895 - val_acc: 0.6917
Epoch 3/100
19/19 [==============================] - 0s 4ms/step - loss: 0.0785 - acc: 0.9867 - val_loss: 0.6752 - val_acc: 0.7250
Epoch 4/100
19/19 [==============================] - 0s 4ms/step - loss: 0.0699 - acc: 0.9900 - val_loss: 0.7123 - val_acc: 0.7333
Epoch 5/100
19/19 [==============================] - 0s 4ms/step - loss: 0.0797 - acc: 0.9917 - val_loss: 0.7257 - val_acc: 0.7250
Epoch 6/100
19/19 [==============================] - 0s 5ms/step - loss: 0.0716 - acc: 0.9933 - val_loss: 0.7709 - val_acc: 0.7167
Epoch 7/100
19/19 [==============================] - 0s 4ms/step - loss: 0.0701 - acc: 0.9967 - val_loss: 0.7970 - val_acc: 0.7417
Epoch 8/100
19/19 [==============================] - 0s 5ms/step - loss: 0.0731 - 

In [85]:
history_d.history.keys()

dict_keys(['loss', 'acc', 'val_loss', 'val_acc'])

In [62]:
#### ------------------------------------------- ####
#### ------------- SHOW CURVES ------------------#### 
#### ------------------------------------------- ####

In [63]:
# Load the TensorBoard notebook extension


In [ ]:
Run tensorboard --logdir "logspath" in terminal